In [254]:
import nltk
from nltk.corpus import brown, words, stopwords, reuters, wordnet, abc, gutenberg, nps_chat
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import re

In [341]:
import csv 
import os
from collections import Counter

In [328]:
# import and read previously created dictionary as csv for abbreviations and expansions of words
def csv2dict(csv_name):
    with open(csv_name, mode='r') as f:
        reader = csv.reader(f)
        next(reader)
        return {rows[1]:rows[2] for rows in reader}
    
    
# create list of unique tokens for a corpus
def unique_words(tokens):
    unique_list = []
    for x in tokens:
        if len(x)>1:
            unique_list.append(x)
    unique_array = np.array(unique_list)
    words = np.unique(unique_array)
    return list(words)

### create correct spelling list to check data against

In [256]:
# create a list of correctly spelled words from various corpi
bw = brown.words()
ww = words.words()
rw = reuters.words()
wnw = wordnet.words()
aw = abc.words()
gw = gutenberg.words()
nw = nps_chat.words()
wordlist_lower = []

for i in bw:
    wordlist_lower.append(i.lower())
    
for i in ww:
    wordlist_lower.append(i.lower())
    
for i in rw:
    wordlist_lower.append(i.lower())
    
for i in wnw:
    wordlist_lower.append(i.lower())

for i in bw:
    wordlist_lower.append(i.lower())
    
for i in aw:
    wordlist_lower.append(i.lower())
    
for i in gw:
    wordlist_lower.append(i.lower())
    
for i in nw:
    wordlist_lower.append(i.lower())
    
test = ' '.join(wordlist_lower)
strip_test = re.sub(r"'s", r'', test)
strip_test = re.sub(r'([^a-zA-Z])', r' ', strip_test)
strip_test = re.sub(r'(\s{2,})', r' ', strip_test)
spell_words = strip_test.split(' ')

In [331]:
initialisms_dict = csv2dict(r'utils/initialisms.csv')

initialisms = list(initialisms_dict.keys())

for i in initialisms:
    wordlist_lower.append(i.lower())
    
word_list = unique_words(spell_words)

### process data to be checked

In [386]:
df = pd.read_csv('train_update_nosw.csv')

In [387]:
df['tweet_tokens'] = df.tweets.str.lower()
tokens = df.tweet_tokens.apply(lambda row: row.split())
word_dict = Counter(tokens.sum())
token_list = list(word_dict.keys())
tokes = unique_words(token_list)

### sort lists of words alphabetically & by length (longest to shortest)

In [396]:
tokes.sort() 
tokes.sort(key=len, reverse=True) 
word_list.sort() 
word_list.sort(key=len, reverse=True) 

### create misspelled list from unique word tokens not present in correctly spelled word corpus

In [399]:
misspelled = []
misspelled = [i for i in tokes if i not in word_list]

In [400]:
corspelled = []
corspelled = [i for i in toke_list if i not in misspelled]

In [405]:
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=5)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
input_term = "memebers"  # misspelling of "members"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2)
# display suggestion term, term frequency, and edit distance
for suggestion in suggestions:
    print(suggestion)

'thisiswhywecanthavenicethings'

In [472]:
segmentations = []
corrections = []
replacement_info = []
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=6)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# # Set max_dictionary_edit_distance to avoid spelling correction
# sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
# dictionary_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_dictionary_en_82_765.txt")
# # term_index is the column of the term and count_index is the
# # column of the term frequency
# sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

for word in misspelled:
    result = sym_spell.word_segmentation(word)
    segmentations.append(result.corrected_string)
    details = ("{}, {}".format(result.distance_sum,
                          result.log_prob_sum))
    replacement_info.append(details)
    suggestions = sym_spell.lookup(word, Verbosity.TOP)
    for suggestion in suggestions:
        corrections.append(suggestion.term)


In [ ]:
segmentations = []
corrections = []
replacement_info = []
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=6)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# # Set max_dictionary_edit_distance to avoid spelling correction
# sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
# dictionary_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_dictionary_en_82_765.txt")
# # term_index is the column of the term and count_index is the
# # column of the term frequency
# sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

for word in misspelled:
    try:
        result = sym_spell.word_segmentation(word)
   
        segmentations.append(result.corrected_string)
#         details = ("{}, {}".format(result.distance_sum,
#                           result.log_prob_sum))
#         replacement_info.append(details)
    except:
        segmentations.append(' ')

#     suggestions = sym_spell.lookup(word, Verbosity.TOP)
#     for suggestion in suggestions:
#         corrections.append(suggestion.term)

In [478]:
segmentations

['this is why we cant have nice things',
 'cant wait to play in minneapolis',
 'give back kali n white account',
 'post apocalyptic flimfl am',
 'that s what friends are for',
 'underground bestsellers',
 'we all heart one direction',
 'keeping the vi gina clean',
 'hawaiian paddle sports',
 'legions track and field',
 'ay hhd jjfjrjjrdjjeks',
 'conditions privation',
 'dead grass and flowers',
 'electrocuted boiling',
 'great british bake off',
 'growing up in colorado',
 'hot teen nsfw porn milf',
 'mother nature network',
 'accidental prophecy',
 'bangladesh affected',
 'infectious diseases',
 'lavender poetry cafe',
 'pretty boy shy f lizzy',
 'reference reference',
 're watching the pilot',
 'this is public health',
 'car accident lawyer',
 'change for the worse',
 'crazy ideas college',
 'deejay empire sound',
 'desolation of sm aug',
 'don t expect nothing',
 'indian perpetrated',
 'intr a generational',
 'jay len e joy bel i gan',
 'not going out in that',
 'online communities'

In [464]:
misspelled

['thisiswhywecanthavenicethings',
 'cantwaittoplayinminneapolis',
 'givebackkalinwhiteaccount',
 'postapocalypticflimflam',
 'thatswhatfriendsarefor',
 'undergroundbestsellers',
 'weallheartonedirection',
 'keepingtheviginaclean',
 'hawaiianpaddlesports',
 'legionstrackandfield',
 'ayhhdjjfjrjjrdjjeks',
 'conditionsprivation',
 'deadgrassandflowers',
 'electrocutedboiling',
 'greatbritishbakeoff',
 'growingupincolorado',
 'hotteennsfwpornmilf',
 'mothernaturenetwork',
 'accidentalprophecy',
 'bangladeshaffected',
 'infectiousdiseases',
 'lavenderpoetrycafe',
 'prettyboyshyflizzy',
 'referencereference',
 'rewatchingthepilot',
 'thisispublichealth',
 'caraccidentlawyer',
 'changefortheworse',
 'crazyideascollege',
 'deejayempiresound',
 'desolationofsmaug',
 'dontexpectnothing',
 'indianperpetrated',
 'intragenerational',
 'jaylenejoybeligan',
 'notgoingoutinthat',
 'onlinecommunities',
 'preseasonworkouts',
 'sidjsjdjekdjskdjd',
 'siteinvestigating',
 'triciaoneillphoto',
 'abbruchsimu

In [458]:
corrections

['notifications',
 'repercussions',
 'billionaires',
 'butterfingers',
 'collectibles',
 'copperfield',
 'demonstration',
 'directions',
 'emergencies',
 'environmental',
 'firefighters',
 'lollapalooza',
 'progressives',
 'realizations',
 'specialness',
 'scattershot',
 'sponsorships',
 'stereotypical',
 'stormtroopers',
 'superintend',
 'university',
 'watermelon',
 'apocalyptic',
 'confident',
 'befriends',
 'consumerist',
 'deployments',
 'dispatchers',
 'fairgrounds',
 'independent',
 'influences',
 'ideographic',
 'intersection',
 'lichfield',
 'mainlines',
 'problematic',
 'remaining',
 'retrospect',
 'screenshots',
 'underpasses',
 'abandoned',
 'activation',
 'airstrikes',
 'anonymous',
 'presuppose',
 'bravissimo',
 'bicyclists',
 'blackburn',
 'bloodhound',
 'foxhunting',
 'brilliantly',
 'parramatta',
 'bulgarians',
 'carpooling',
 'chinchilla',
 'commenter',
 'crematoria',
 'dardanelles',
 'daughter',
 'dilution',
 'dominican',
 'episodes',
 'equipments',
 'everything',
 '